<a href="https://colab.research.google.com/github/git4sudo/CS583B_project/blob/main/DL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path='/content/drive/MyDrive/GYAFC_Corpus'

In [3]:
!dir

drive  sample_data


In [4]:
!dir drive/MyDrive/GYAFC_Corpus/

Entertainment_Music  Family_Relationships  README.txt


In [5]:
def show(path):
  readme=open(path,'r')
  print(readme.read())

In [6]:
import pandas as pd

In [7]:
!dir 'drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train'

formal	informal


In [8]:
show('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/formal')

The In-Laws movie isn't a holiday movie, but it's okay.
I don't think that page gave me viruses.
I watch it everyday, my favorite charachter is Inuasha.
Funbrain.com and runescape.com are great for family fun.
He was on the Late Night show with Conan O'Brien and seemed gay.
Mel Gibson is a strong believed of God.
My exams are not over yet because I still have one left.
Sky Kids 3D: game over.
His mom was a wafer so long.
I am entirely off or does that help.
Her name is Jessica.
www.us.imdb.com is probably one of your best bets.
Titanic is my favorite movie and I cry when I see it.
It gives us something to do.
He is debonair, or perhaps more than that.
Many people enjoy the MLB.
I do not possess that model, but I am partial to my Creative Zen Micro
I viewed it and I believe it is a quality program.
She is a poor vocalist.
He is very attractive.
This is an example of something someone, who is age 20 and married, would do.
What am I getting paid for doing this?
Listen to what the first pe

In [9]:
df_test=pd.DataFrame()

In [10]:
df_test['informal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/test/informal.ref0',sep='\t')

In [11]:
df_test['formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/test/formal',sep='\t')

In [12]:
df_train=pd.DataFrame()

In [13]:
df_train['informal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/informal',sep='\t')

In [14]:
df_train['formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/formal',sep='\t')

In [15]:
df_train.head(100)

,informal,formal
0,that page did not give me viroses(i think),I don't think that page gave me viruses.
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.
...,...,...
95,who cares what clothes seventeen magazine says...,Who cares about what clothes that seventeen ma...
96,dude who sings for the cure,The man who sings for The Cure.
97,well a geek to me would have to be someone i c...,"Well, a geek, to me, would have to be someone ..."
98,"If nothing, his last name is Japanese ;)",His last name is Japanese.


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52594 entries, 0 to 52593
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   informal  52594 non-null  object
 1   formal    52592 non-null  object
dtypes: object(2)
memory usage: 821.9+ KB


In [17]:
#seq2seq model

In [18]:
!pip install tensorflow

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
all_texts = pd.concat([df_train['formal'], df_train['informal'], df_test['formal'], df_test['informal']]).astype(str)


In [21]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)



In [22]:
train_formal_seqs = tokenizer.texts_to_sequences(df_train['formal'].astype(str))
train_informal_seqs = tokenizer.texts_to_sequences(df_train['informal'].astype(str))
test_formal_seqs = tokenizer.texts_to_sequences(df_test['formal'].astype(str))
test_informal_seqs = tokenizer.texts_to_sequences(df_test['informal'].astype(str))



In [23]:
train_formal_seqs = pad_sequences(train_formal_seqs, padding='post')
train_informal_seqs = pad_sequences(train_informal_seqs, padding='post')
test_formal_seqs = pad_sequences(test_formal_seqs, padding='post')
test_informal_seqs = pad_sequences(test_informal_seqs, padding='post')


In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256
units = 1024

# Define Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Define Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Train the model
model.fit([train_informal_seqs, train_formal_seqs], train_formal_seqs, batch_size=64, epochs=16, validation_split=0.2)


Epoch 1/16
658/658 [==============================] - 182s 254ms/step - loss: 1.7860 - val_loss: 1.4237
Epoch 2/16
658/658 [==============================] - 168s 255ms/step - loss: 1.1880 - val_loss: 1.0442
Epoch 3/16
658/658 [==============================] - 169s 256ms/step - loss: 0.9308 - val_loss: 0.8673
Epoch 4/16
658/658 [==============================] - 168s 255ms/step - loss: 0.7955 - val_loss: 0.7652
Epoch 5/16
658/658 [==============================] - 169s 256ms/step - loss: 0.7067 - val_loss: 0.6928
Epoch 6/16
658/658 [==============================] - 168s 256ms/step - loss: 0.6395 - val_loss: 0.6376
Epoch 7/16
658/658 [==============================] - 174s 264ms/step - loss: 0.5853 - val_loss: 0.5958
Epoch 8/16
658/658 [==============================] - 169s 256ms/step - loss: 0.5406 - val_loss: 0.5586
Epoch 9/16
658/658 [==============================] - 168s 256ms/step - loss: 0.5028 - val_loss: 0.5281
Epoch 10/16
658/658 [==============================] - 169s 257m

In [25]:
!pip install nltk pyter3

In [28]:
def generate_with_seq2seq(model, tokenizer, sentence, max_length=50):
    # Tokenize the input sentence
    sequence = tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Encode the input
    # Assuming your model has a separate encoder and decoder
    encoder_output, state_h, state_c = model.encoder(sequence)
    encoder_states = [state_h, state_c]

    # Decode the encoded input
    # Start with a sequence containing only the start-of-sentence token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']  # assuming you have a start token

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = model.decoder.predict([target_seq] + encoder_states)

        # Sample a token and add the corresponding word to the decoded sentence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop token.
        if (sampled_word == '<end>' or len(decoded_sentence) > max_length):
            stop_condition = True

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        encoder_states = [h, c]

    return decoded_sentence


In [ ]:
from transformers import pipeline
import pyter
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# Load the Hugging Face model
style_transfer_hf = pipeline("text2text-generation", model="prithivida/informal_to_formal_styletransfer")

# Function to evaluate a single model
def evaluate_model(model, test_data):
    bleu_scores = []
    ter_scores = []

    for informal, formal in test_data:
        # Generate formal style text
        if model == 'hf':
            result = style_transfer_hf(informal)[0]['generated_text']
        else:
            # Add code to generate text using your TensorFlow seq2seq model
            result = generate_with_seq2seq(model, informal)

        # Calculate BLEU score
        reference = [formal.split()]
        candidate = result.split()
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

        # Calculate TER score
        ter_score = pyter.ter(candidate, reference[0])
        ter_scores.append(ter_score)

    # Calculate average scores
    avg_bleu = np.mean(bleu_scores)
    avg_ter = np.mean(ter_scores)

    return avg_bleu, avg_ter
# Assuming df_test is your DataFrame with 'formal' and 'informal' columns
test_dataset = list(zip(df_test['informal'], df_test['formal']))

# hf_bleu, hf_ter = evaluate_model('hf', test_dataset)
# seq2seq_bleu, seq2seq_ter = evaluate_model(your_seq2seq_model, test_dataset)

# Evaluate Hugging Face model
hf_bleu, hf_ter = evaluate_model('hf', test_dataset)

# Evaluate TensorFlow seq2seq model
# seq2seq_bleu, seq2seq_ter = evaluate_model(your_seq2seq_model, test_dataset)

# Print results
print("Hugging Face Model - BLEU:", hf_bleu, "TER:", hf_ter)
# print("TensorFlow Seq2Seq Model - BLEU:", seq2seq_bleu, "TER:", seq2seq_ter)
